# TP3 - Grupo 20
###  Dezembro, 4, 2024

Afonso Martins Campos Fernandes - A102940

Luís Filipe Pinheiro Silva - A105530

## Problema 1

# Algoritmo Estendido de Euclides (EXA)

O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes \(a, b > 0\) e devolve inteiros \(r, s, t\) tais que:

$$
a \cdot s + b \cdot t = r
$$

e 

$$
r = \gcd(a, b)
$$

Além das variáveis \(r, s, t\), o código requer três variáveis adicionais \(r', s', t'\), que representam os valores de \(r, s, t\) no “próximo estado”.

## **Algoritmo**

```plaintext
INPUT a, b
assume a > 0 and b > 0
r, r', s, s', t, t' = a, b, 1, 0, 0, 1
while r' != 0
  q = r div r'
  r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t'
OUTPUT r, s, t



### a) Construa um SFOTS usando BitVector’s de tamanho $$n$$ que descreva o comportamento deste programa.  Considere estado de erro quando $$\,r=0\,$$ ou alguma das variáveis atinge o “overflow”.

# Objetivo


O objétivo do modelo:
1. **Cálculo do Máximo Divisor Comum (MDC)** entre dois inteiros `a` e `b` utilizando o algoritmo estendido de Euclides.
2. Acompanhar os estados intermédios (`r`, `r_linha`, `s`, `s_linha`, `t`, `t_linha`) e as transições associadas.
3. Identificação das condições de erro:
    - Quando o valor de `r` é igual a zero;
    - Quado ocorre **overflow** nos registos.

In [5]:
from pysmt.shortcuts import *
from pysmt.typing import BVType, INT

def gera_estado(vars, s, i, n):
    estado = {var: Symbol(f"{var}!{s}{i}", BVType(n)) for var in vars}
    estado['pc'] = Symbol(f"pc!{s}{i}", INT)
    return estado 

def estado_inicial(estado, a, b, n):
    return And(
        Equals(estado['r'], a),
        Equals(estado['r_linha'], b),
        Equals(estado['s'], BV(1, n)),
        Equals(estado['s_linha'], BV(0, n)),
        Equals(estado['t'], BV(0, n)),
        Equals(estado['t_linha'], BV(1, n)),
        Equals(estado['q'], BV(0, n)),
        BVUGT(a, BV(0, n)),
        BVUGT(b, BV(0, n)),
        Equals(estado['pc'], Int(0))
    )

def erro(estado):
    return Equals(estado['pc'], Int(3))

def transicao(atual, prox):
    n = atual['r'].symbol_type().width

    def transP():
        return And(
            Or(Equals(atual['pc'], Int(0)), Equals(atual['pc'], Int(1))),
            Not(Equals(atual['r_linha'], BV(0, n))),
            Equals(prox['q'], BVUDiv(atual['r'], atual['r_linha'])),
            Equals(prox['pc'], Int(1) - atual['pc']),
            Equals(prox['r'], atual['r_linha']),
            Equals(prox['r_linha'], BVSub(atual['r'], BVMul(atual['q'], atual['r_linha']))),
            Equals(prox['s'], atual['s_linha']),
            Equals(prox['s_linha'], BVSub(atual['s'], BVMul(atual['q'], atual['s_linha']))),
            Equals(prox['t'], atual['t_linha']),
            Equals(prox['t_linha'], BVSub(atual['t'], BVMul(atual['q'], atual['t_linha'])))
        )

    def erro0():
        return And(
            Equals(atual['pc'], Int(1)),
            Equals(atual['r'], BV(0, n)),
            Equals(prox['pc'], Int(3))
        )

    def overflow():
        overflow_checks = [
            BVUGT(atual[var], BV(2**n - 1, n))
            for var in ['r', 'r_linha', 's', 's_linha', 't', 't_linha', 'q']
        ]
        return And(
            Equals(atual['pc'], Int(1)),
            Or(*overflow_checks),
            Equals(prox['pc'], Int(3))
        )

    def estado_final():
        return And(
            Equals(atual['pc'], Int(1)),
            Equals(atual['r_linha'], BV(0, n)),
            Equals(prox['pc'], Int(2)),
            *(Equals(prox[var], atual[var]) for var in ['r', 'r_linha', 's', 's_linha', 't', 't_linha', 'q'])
        )
 
    return Or(
        transP(),
        erro0(),
        overflow(),
        estado_final()
    )

def gera_trace(vars, estado_inicial, transicao, erro, n, N):
    with Solver(name="z3") as solver:
        S = [gera_estado(vars, 'S', i, n) for i in range(N+1)]
        a = Symbol('a_input', BVType(n))  # Nome único para evitar conflitos
        b = Symbol('b_input', BVType(n))  # Nome único para evitar conflitos
        I = estado_inicial(S[0], a, b, n)
        traces = [transicao(S[i], S[i+1]) for i in range(N)]

        estado_final = Equals(S[N]['pc'], Int(2))

        if solver.solve([I, And(traces), estado_final]):
            model = solver.get_model()
            for i in range(N+1):
                print(f"Estado S{i}:")
                for v in S[i]:
                    if S[i][v] in model:
                        valor = model[S[i][v]].constant_value()
                        print(f"        {v} = {valor}")
        else:
            print("Nenhum Modelo Encontrado!")

vars = ['r', 'r_linha', 's', 's_linha', 't', 't_linha', 'q']
gera_trace(vars, estado_inicial, transicao, erro, 8, 8)

Estado S0:
        r = 64
        r_linha = 136
        s = 1
        s_linha = 0
        t = 0
        t_linha = 1
        q = 0
        pc = 0
Estado S1:
        r = 136
        r_linha = 64
        s = 0
        s_linha = 1
        t = 1
        t_linha = 0
        q = 0
        pc = 1
Estado S2:
        r = 64
        r_linha = 136
        s = 1
        s_linha = 0
        t = 0
        t_linha = 1
        q = 2
        pc = 0
Estado S3:
        r = 136
        r_linha = 48
        s = 0
        s_linha = 1
        t = 1
        t_linha = 254
        q = 0
        pc = 1
Estado S4:
        r = 48
        r_linha = 136
        s = 1
        s_linha = 0
        t = 254
        t_linha = 1
        q = 2
        pc = 0
Estado S5:
        r = 136
        r_linha = 32
        s = 0
        s_linha = 1
        t = 1
        t_linha = 252
        q = 0
        pc = 1
Estado S6:
        r = 32
        r_linha = 136
        s = 1
        s_linha = 0
        t = 252
        t_linha = 1
      

### 1) Estrutura:

### 1.1) Componentes:

- Variáveis de Estado:
    
    - `r` e `r_linha`: Variáveis de Estado para o cálculo do MDC;
    
    - `s`, `s_linha`, `t`, `t_linha`: Coeficientes de Bèzout;  

    - `q`: Quoeficiente da Divisão;

    - `pc`: "Program Counter"

### 1.2) Funções Principais:

- `def gera_estados(vars, s, i, n)`:

Esta função gera um novo estado do sistema com variáveis simbólicas, sendo os seus parametros com uma lista de variáveis, prefixo do estado, indice e o número de bits.

- `def estado_inicial(estado, a, b, n)`:

Esta função define as condições iniciais do sistema, inicializando todas as variáveis com os seus valores iniciais. 

Estabelece a Pré-condição:

### (a > 0 && b > 0)

- `def transicao(atual, prox):`

A função representada define as regras para a transição de estados, implementa a lógica principal do algoritmo e inclui verificações de erro caso `r' == 0` e em caso de _overflow_

- `def erro0():`

Função que verifica se o programa se situa no estado 1, pc = 1, detecta se a var. `r` chegou a zero de forma correta e quando estas condições são satisfeitas o programa move-se para o estado de erro, pc = 3.

O erro pode ocorrer em dois casos:
1. O algoritmo atinge uma condição inválida como `r' = 0`;
2. A sequência de divisões nao segue o padrão do algoritmo de Euclides.


- `def overflow():`

A função implementa a verificação da existência overflow nas variàveis do sistema:

1. Verificações:

- É criada uma lista de verificações para cada variável;
- Utiliza ´BVUGT´(BitVector Unsigned Greater Than) para comparar o valor máximo permitido(2^n - 1, onde n é o número de bits)

2. Condições para a função ser ativada:

- Verifica se o programa se encontra no estado 1;
- Se alguma var. execeder o valor máximo;
- Caso estas condições sejam admitidas o programa move para o estado de erro, pc = 3.

Esta função é crucial para garantir a integridade dos cálculos, previne o comportamento indefinido por overflow e mantém a correção do algorítmo.



### 2) Verificação Formal:

### 2.1) Propriedades Verificadas:

1. Correção:

- Verifica se o algoritmo termina no estado correto;
- Garante que o MDC é calculado corretamente;

2. Segurança: 

- Detecta a exitência de _overflow_ nas operações;
- Previne a divisão por zero;
- Identifica estados de erro;

### 2.2) Estados do Sistema:

No programa apresentamos 4 estados, sendo:
- *Estado 0*: Estado Inicial, todas as variáveis são definidas corretamente;

- *Estado 1*: Estado intermédio, faz os cálculos aritméticos do algoritmo;

- *Estado 2*: Estado Final, quando o ´r == 0´ o programa passa para este estado para finalizar o seu processo;

- *Estado 3*: Estado de Erro, caso alguma propriedade de erro seja verficada o programa desloca do `Estado 1` para este estado para finalizar o seu processo;

### 3) Implementação
      
```
vars = ['r', 'r_linha', 's', 's_linha', 't', 't_linha', 'q']
gera_trace(vars, estado_inicial, transicao, erro, 8, 8) 
```

Para testarmos se o sistema está funcionar corretamente utilizamos estas duas linhas para o mesmo ser testado. A primeira é apenas uma lista das variáveis a serem utilizadas(referidas anteriormente). A segunda chama a função `gera_trace` onde são dados os parametros: lista de variáveis(`vars`), o estado inicial(`estado_inicial`), a função transição(`transicao`), a função estado de erro(`erro`),o número de bits(8) e o número de passos a serem efetuados(8). 

A função `gera_trace(vars, estado_inicial, transicao, erro, 8, 8)` ao ser executado devolve os valores de todas as variáveis em cada estado, transições de cada estado e detecta caso exista condições de erro ou de sucesso.





### b) Prove, usando a metodologia dos invariantes interpolantes, que o modelo nunca atinge o estado de erro

In [6]:
def invariante_seguranca(estado, n):
    """Invariante: pc ≠ 3"""
    return Not(Equals(estado['pc'], Int(3)))

def invariante_valores_positivos(estado):
    """Invariante: r > 0 ∧ r_linha > 0"""
    return And(
        BVUGT(estado['r'], BV(0, estado['r'].symbol_type().width)),
        BVUGT(estado['r_linha'], BV(0, estado['r'].symbol_type().width))
    )

def invariante_limites(estado, n):
    """Invariante: ∀var ∈ {r,r_linha,s,s_linha,t,t_linha,q}: var < 2^n"""
    limite = BV(2**n - 1, n)
    checks = []
    for var in ['r', 'r_linha', 's', 's_linha', 't', 't_linha', 'q']:
        checks.append(BVULE(estado[var], limite))
    return And(checks)

def interpolante_pc0_pc1(estado, n):
    """Interpolante entre pc=0 e pc=1"""
    return And(
        BVUGT(estado['r'], BV(0, n)),
        BVUGT(estado['r_linha'], BV(0, n)),
        BVUGE(estado['r'], estado['r_linha'])
    )

def interpolante_pc1_pc2(estado, n):
    """Interpolante entre pc=1 e pc=2"""
    return And(
        BVUGT(estado['r'], BV(0, n)),
        Equals(estado['r_linha'], BV(0, n))
    )

def verifica_propriedades(n, N):
    with Solver(name="z3") as solver:
        # Gerar estados
        vars = ['r', 'r_linha', 's', 's_linha', 't', 't_linha', 'q']
        S = [gera_estado(vars, 'S', i, n) for i in range(N+1)]
        
        # Símbolos de entrada
        a = Symbol('a_input', BVType(n))
        b = Symbol('b_input', BVType(n))
        
        # Estado inicial
        I = estado_inicial(S[0], a, b, n)
        
        # Transições
        T = [transicao(S[i], S[i+1]) for i in range(N)]
        
        # Invariantes e interpolantes para cada estado
        invariantes = []
        for estado in S:
            invariantes.extend([
                invariante_seguranca(estado, n),
                invariante_valores_positivos(estado),
                invariante_limites(estado, n)
            ])
            
            # Adicionar interpolantes baseados no pc
            if Equals(estado['pc'], Int(0)):
                invariantes.append(interpolante_pc0_pc1(estado, n))
            elif Equals(estado['pc'], Int(1)):
                invariantes.append(interpolante_pc1_pc2(estado, n))
        
        # Verificar se é possível atingir o estado de erro
        erro_atingivel = Or([erro(s) for s in S])
        
        # Adicionar todas as condições ao solver
        solver.add_assertion(I)
        solver.add_assertion(And(T))
        solver.add_assertion(And(invariantes))
        solver.add_assertion(erro_atingivel)
        
        # Verificar se existe solução
        if solver.solve():
            print("INSEGURO: Estado de erro pode ser atingido!")
            model = solver.get_model()
            # Mostrar o contraexemplo
            for i, estado in enumerate(S):
                print(f"\nEstado {i}:")
                for var in estado:
                    if estado[var] in model:
                        print(f"  {var} = {model[estado[var]].constant_value()}")
        else:
            print("O SISTEMA É SEGURO: Estado de erro não pode ser atingido!")
            print("Propriedades verificadas:")
            print("1. pc ≠ 3 (invariante de segurança)")
            print("2. r > 0 ∧ r_linha > 0 (invariante de valores positivos)")
            print("3. Todas as variáveis estão dentro dos limites")
            print("4. Interpolantes entre estados mantidos")

# Executar a verificação
verifica_propriedades(8, 8)

O SISTEMA É SEGURO: Estado de erro não pode ser atingido!
Propriedades verificadas:
1. pc ≠ 3 (invariante de segurança)
2. r > 0 ∧ r_linha > 0 (invariante de valores positivos)
3. Todas as variáveis estão dentro dos limites
4. Interpolantes entre estados mantidos


### 1) Invariantes do sistema:

### 1.1) Invariante para o estado de erro:

A função `invariante_segurança(estado, n)` garante que o contadoe do programa, `pc`, nunca atinja o estado de erro/`pc = 3`. Pode ser representado no formato:

$$
I_{\text{segurança}} : pc \neq 3
$$

### 1.2) Invariantes de valores positivos:

A função:
```py
def invariante_valores_positivos(estado):
    return And(
        BVUGT(estado['r'], BV(0, estado['r'].symbol_type().width)),
        BVUGT(estado['r_linha'], BV(0, estado['r'].symbol_type().width))
    )
```
garante que as variáveis `r` e `r_linha` permaneçam sempre positivas. Reperesentado no formato:

$$
I_{\text{valores positivos}} : r > 0 \land r_{\text{linha}} > 0
$$


### 1.3) Invariante de limites:

A função:
```py
def invariante_limites(estado, n):
    limite = BV(2**n - 1, n)
    checks = []
    for var in ['r', 'r_linha', 's', 's_linha', 't', 't_linha', 'q']:
        checks.append(BVULE(estado[var], limite))
    return And(checks)
```
faz com que todas as variáveis utilizadas têm todas limites definidos por um número de bits(n)

$$
I_{\text{limites}} : \forall \text{var} \in \{r, r_{\text{linha}}, s, s_{\text{linha}}, t, t_{\text{linha}}, q\} : \text{var} < 2^n
$$

### 2) Interpolantes:

### 2.1) Interpolante entre PC = 0 e PC = 1:

A partir da função `interpolante_pc0_pc1(estado, n)` são definidas condições que devem se mantidas do estado inicial para o estado intermédio.

$$
\phi_{0 \to 1} : r > 0 \land r_{\text{linha}} > 0 \land r \geq r_{\text{linha}}
$$

### 2.2) Interpolante entre PC = 1 e PC = 2:

A função:

```py
def interpolante_pc1_pc2(estado, n):
    """Interpolante entre pc=1 e pc=2"""
    return And(
        BVUGT(estado['r'], BV(0, n)),
        Equals(estado['r_linha'], BV(0, n))
    )
```

estabelece as condições para a existir a transição do estado 1 para o estado final, pc = 2, para a finalizar o processo do sistema.


$$
\phi_{1 \to 2} : r > 0 \land r_{\text{linha}} = 0
$$


### 3) Verificação:

Para a verficação do sistema, são feitos os seguintes passos:

1. Iniciação:

São gerados diferentes estados, definidos símbolos de entrada e é estabelecido o estado inicial.

2. Transições:

São definidas as transições, os invariantes e os interpolantes são aplicados e de seguida é verificado se o estado de erro é possível ser obtido ou não.

3. Implementação da Verificação:

Utilizamos a função `verifica_propriedades(n, N)` que nos mostra de forma simples e eficaz se o sistema é seguro onde o estado de erro nunca é atingido ou vice-versa.


